In [1]:
import numpy as np
import pandas as pd
import napari
import tifffile
import skimage as ski
import scipy.ndimage as ndi
import glob
import plotly.express as px
import cellpose.models as models
import matplotlib.pyplot as plt
import cv2
import dask
import cellpose.models as models

In [22]:
def shrink_labels(label_image, shrinkage=3):
    """
    This function shrinks the regions in a labeled image using morphological erosion.

    Parameters:
    label_image (numpy.ndarray): A 2D array where each unique non-zero value represents a unique object/region.
    shrinkage (int, optional): The size of the structuring element used for the erosion operation. Default is 3.  This default shrinks by 1 pixel in each direction.

    Returns:
    shrunken_label_image (numpy.ndarray): A 2D array similar to label_image but with all regions shrunken.

    How it works:
    1. It first creates a circular structuring element of the specified size.
    2. It then initializes an empty image of the same size as the input image.
    3. It calculates the properties of each labeled region in the input image using skimage.measure.regionprops.
    4. For each region, it extracts the region from the original label image, erodes it using the structuring element, and then updates the corresponding region in the shrunken_label_image.
    5. The erosion operation shrinks the region by removing pixels from its boundary.
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (shrinkage, shrinkage))

    shrunken_label_image = np.zeros_like(label_image)
    regions = ski.measure.regionprops(label_image)

    for region in regions:
        lbl = region.label
        minr, minc, maxr, maxc = region.bbox

        minr = max(minr - 1, 0)
        minc = max(minc - 1, 0)
        maxr = min(maxr + 1, label_image.shape[0])
        maxc = min(maxc + 1, label_image.shape[1])

        # Extract the region from the original label image
        region_mask = label_image[minr:maxr, minc:maxc] == lbl

        # Erode the region
        eroded_region_mask = cv2.erode(region_mask.astype(np.uint8), kernel)

        # Update the shrunken_label_image
        shrunken_label_image[minr:maxr, minc:maxc][eroded_region_mask == 1] = lbl

    return shrunken_label_image

In [60]:
def remove_objects(label_image, area_min, area_max):
    """
    This function removes objects from a labeled image based on their area.

    Parameters:
    label_image (numpy.ndarray): A 2D array where each unique non-zero value represents a unique object/region.
    area_min (int): The minimum area threshold. Objects with area less than this will be removed.
    area_max (int): The maximum area threshold. Objects with area more than this will be removed.

    Returns:
    cleaned_label_image (numpy.ndarray): A 2D array similar to label_image but with small and large objects removed.

    How it works:
    1. It first calculates the properties of each labeled region in the input image using skimage.measure.regionprops.
    2. It then creates a boolean mask of the same size as the input image. This mask is True for pixels belonging to regions that are too small or too large.
    3. Finally, it applies this mask to the input image, setting the labels of the small and large regions to zero, effectively removing them from the image.
    """
    # Get properties of labeled regions
    props = ski.measure.regionprops(label_image)

    # Create a boolean mask where True indicates a region is too small or too large
    object_mask = np.zeros_like(label_image, dtype=bool)
    for prop in props:
        if prop.area < area_min or prop.area > area_max:
            object_mask[label_image == prop.label] = True

    # Apply the mask to the label image
    cleaned_label_image = np.where(object_mask, 0, label_image)

    return cleaned_label_image

In [4]:
viewer = napari.Viewer()

In [2]:
model = models.Cellpose(gpu=True, model_type='cyto')

# HOMEWORK

## Part 1:  Finding a good diameter

There is a file in our files folder:  chamber1KLF5in488SOX8in647.tif.  Open it up and take a look in napari.  

In [5]:
img = ski.io.imread('files/chamber1KLF5in488SOX8in647.tif')
img.shape

(6144, 6144, 3)

In [6]:
viewer.add_image(img, channel_axis=2)

[<Image layer 'Image' at 0x138a3c5de10>,
 <Image layer 'Image [1]' at 0x138b01a4a90>,
 <Image layer 'Image [2]' at 0x138b01d6da0>]

Now use your cursor in napari to guess a good diameter, and try segmenting the cells with cellpose, you will have to use the "channels" and "channel_axis" parameters.  Remember to choose one of the channels for the nucleus, and another for the cytosol.

In [55]:
cells, flows, styles, diams = model.eval(img, diameter=300, channels=[2,3], channel_axis=2)
viewer.add_labels(cells, name='Diameter only')

<Labels layer 'Diameter only' at 0x1393c196ad0>

## Part 2:  Getting more cells

Even with a good choice of diameter, and a good choice for the channels, you will find cellpose is not segmenting very many of the cells.  Next try to adjust "cellprob_threshold" and "flow_threshold" to get more cells segmented.  If you get some tiny slivers of garbage, don't worry we'll take care of those later.

In [63]:
cells, flows, styles, diams = model.eval(img, diameter=300, channels=[2,3], channel_axis=2, cellprob_threshold=-2, flow_threshold=1.0)
viewer.add_labels(cells, name='Cellprob and flow')


<Labels layer 'Cellprob and flow [1]' at 0x138ae2b7ee0>

## Part 3:  Cleaning up the segmentation

Now we want to do two things:  first clean up the garbage using the "min_size" parameter in model.eval.

In [64]:
filtered_cells = remove_objects(cells, 20000, 10000000000)
viewer.add_labels(cells, name='Cellprob and flow size filtered')

<Labels layer 'Cellprob and flow size filtered [1]' at 0x138ae2b64d0>

Notice that our object boundaries usually stretch out beyond our objects.  This is because cellpose downsamples by so much when it does its segmentation, it effectively blurs.  To correct for this, and only have our segmentation cover where the cell is, we can shrink the label image using the shrink_labels function we've used before.  Play with different values of shrinkage until you think it looks good.

In [65]:
shrunk_cells = shrink_labels(filtered_cells, shrinkage=40)
viewer.add_labels(shrunk_cells, name='Shrunk cells')

<Labels layer 'Shrunk cells [1]' at 0x138ae4580a0>